# Construct Master Dataset by linking different tables in MIMIC-IV-ED

Note:  MIMIC-ED should be downloaded at “data/ed" folder


## Prepare Python library and raw data

In [21]:
import argparse
from helpers import *
import dask.dataframe as dd
from medcode_utils import commorbidity, extract_icd_list

In [22]:
# import tensorflow as tf
# print(tf.__version__)

In [23]:
from dataset_path import mimic_iv_path, output_path

In [24]:
## Defining auxillary parameters
## Assume that "ed" directory placed under "mimic_iv_path"
## ZJ: Make this assumption bold
mimic_iv_core_path = os.path.join(mimic_iv_path, 'core')
mimic_iv_hosp_path = os.path.join(mimic_iv_path , 'hosp')   
mimic_iv_icu_path = os.path.join(mimic_iv_path, 'icu')
mimic_iv_ed_path = os.path.join(mimic_iv_path, 'ed')

icu_filename_dict = {"chartevents":"chartevents.csv","datetimeevents":"datetimeevents.csv","d_items":"d_items.csv","icustays":"icustays.csv","inputevents":"inputevents.csv","outputevents":"outputevents.csv","procedureevents":"procedureevents.csv"}
core_filename_dict = {"patients":"patients.csv", "admissions":"admissions.csv", "transfers":"transfers.csv"}
hosp_filename_dict = {"d_hcpcs":"d_hcpcs.csv","d_icd_diagnoses":"d_icd_diagnoses.csv","d_labitems":"d_labitems.csv","emar":"emar.csv","hcpcsevents":"hcpcsevents.csv","microbiologyevents":"microbiologyevents.csv","poe":"poe.csv","prescriptions":"prescriptions.csv","services":"services.csv","diagnoses_icd":"diagnoses_icd.csv","d_icd_procedures":"d_icd_procedures.csv","drgcodes":"drgcodes.csv","emar_detail":"emar_detail.csv","labevents":"labevents.csv","pharmacy":"pharmacy.csv","poe_detail":"poe_detail.csv","procedures_icd":"procedures_icd.csv"}
ed_filename_dict = {'diagnosis':'diagnosis.csv', 'edstays':'edstays.csv',  'medrecon':'medrecon.csv',  'pyxis':'pyxis.csv',  'triage':'triage.csv',  'vitalsign':'vitalsign.csv'}

complaint_dict = {"chiefcom_chest_pain" : "chest pain", "chiefcom_abdominal_pain" : "abdominal pain|abd pain", 
"chiefcom_headache" : "headache|lightheaded", "chiefcom_shortness_of_breath" : "breath", "chiefcom_back_pain" : "back pain", "chiefcom_cough" : "cough", 
"chiefcom_nausea_vomiting" : "nausea|vomit", "chiefcom_fever_chills" : "fever|chill", "chiefcom_syncope" :"syncope", "chiefcom_dizziness" : "dizz"}

## Defining health utilization timerange parameters in days
icu_transfer_timerange = 12 # hours
next_ed_visit_timerange = 3
# past_ed_visits_timerange = 365
# past_admissions_timerange = 365 
# past_icu_visits_timerange = 365

## Experimenting with Dask Library

In [25]:
# This is everything to test our dask library

In [26]:
# with open(os.path.join(mimic_iv_icu_path, icu_filename_dict['chartevents'])) as f:
#     print(sum(1 for line in f))

In [27]:
# df_charttables = dd.read_csv(os.path.join(mimic_iv_icu_path, icu_filename_dict['chartevents']), dtype={'value': 'float64',
#        'valuenum': 'float64',
#        'valueuom': 'object'})
df_charttables = dd.read_csv(os.path.join(mimic_iv_icu_path, icu_filename_dict['chartevents']), dtype={
    'subject_id': 'float64',
    'hadm_id': 'float64',
    'stay_id': 'float64',
    'charttime': 'object',
    'storetime': 'object',
    'itemid': 'float64',
    'value': 'float64',
    'valuenum': 'float64',
    'valueuom': 'object',
    'warning': 'float64'
})
print(df_charttables.columns)

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'storetime', 'itemid',
       'value', 'valuenum', 'valueuom', 'warning'],
      dtype='object')


In [28]:
print(len(df_charttables.partitions[0]))
print(len(df_charttables.partitions[1]))

739007
739805


In [29]:
df_test = df_charttables.drop('valueuom', axis=1)
df_test.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'storetime', 'itemid',
       'value', 'valuenum', 'warning'],
      dtype='object')

In [30]:
df_test.dtypes

subject_id    float64
hadm_id       float64
stay_id       float64
charttime      object
storetime      object
itemid        float64
value         float64
valuenum      float64
warning       float64
dtype: object

In [31]:
df_test['charttime'] = dd.to_datetime(df_test['charttime'])
df_test['storetime'] = dd.to_datetime(df_test['storetime'])

In [32]:
df_test.dtypes

subject_id           float64
hadm_id              float64
stay_id              float64
charttime     datetime64[ns]
storetime     datetime64[ns]
itemid               float64
value                float64
valuenum             float64
warning              float64
dtype: object

In [33]:
df_test.partitions[365].tail()

ValueError: could not convert string to float: 'SR (Sinus Rhythm)'

In [ ]:
len(df_test['value'].astype('float64'))

In [34]:
df_charttables.npartitions

466

In [35]:
df_charttables.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10003700.0,28623837.0,30600691.0,2165-04-24 05:10:00,2165-04-24 05:11:00,228236.0,0.0,0.0,NaN,0.0
1,10003700.0,28623837.0,30600691.0,2165-04-24 05:12:00,2165-04-24 05:14:00,225067.0,0.0,0.0,NaN,0.0
2,10003700.0,28623837.0,30600691.0,2165-04-24 05:12:00,2165-04-24 05:14:00,225070.0,1.0,1.0,NaN,0.0
3,10003700.0,28623837.0,30600691.0,2165-04-24 05:12:00,2165-04-24 05:14:00,225076.0,1.0,1.0,NaN,0.0
4,10003700.0,28623837.0,30600691.0,2165-04-24 05:12:00,2165-04-24 05:14:00,225078.0,1.0,1.0,NaN,0.0


In [36]:
df_test = df_charttables.compute()

ValueError: could not convert string to float: 'Aspen'

In [37]:
import psutil
psutil.virtual_memory()

svmem(total=404358152192, available=359514128384, percent=11.1, used=37289144320, free=197063168000, active=141293666304, inactive=46398521344, buffers=606470144, cached=169399369728, shared=5684744192, slab=3836624896)

In [38]:
pt = df_charttables.partitions[365]
pt.head()

ValueError: could not convert string to float: 'SR (Sinus Rhythm)'

In [39]:
df_edstays = read_edstays_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['edstays']))

In [40]:
df_edstays.dtypes

subject_id             int64
hadm_id              float64
stay_id                int64
intime        datetime64[ns]
outtime       datetime64[ns]
dtype: object

## Load raw data tables through pandas library

In [56]:
## Reading main tables
df_edstays = read_edstays_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['edstays']))
df_patients = read_patients_table(os.path.join(mimic_iv_core_path, core_filename_dict['patients']))
df_admissions = read_admissions_table(os.path.join(mimic_iv_core_path, core_filename_dict["admissions"]))
df_icustays = read_icustays_table(os.path.join(mimic_iv_icu_path, icu_filename_dict['icustays']))
df_triage = read_triage_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['triage']))
df_vitalsign = read_vitalsign_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['vitalsign']))
df_pyxis = read_pyxis_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['pyxis']))
df_medrecon = read_pyxis_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['medrecon']))

## Read data here for ICD.
df_diagnoses = read_diagnoses_table(os.path.join(mimic_iv_hosp_path, hosp_filename_dict['diagnoses_icd']))
# df_diagnoses = read_diagnoses_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['diagnosis']))

In [57]:
print(df_diagnoses.shape[0])
print(df_diagnoses.head(10))

5280351
   subject_id   hadm_id  seq_num icd_code  icd_version
0    15734973  20475282        3     2825            9
1    15734973  20475282        2    V0251            9
2    15734973  20475282        5     V270            9
3    15734973  20475282        1    64891            9
4    15734973  20475282        4    66481            9
5    11442057  21518990        1    65971            9
6    11442057  21518990        2    64231            9
7    11442057  21518990        6     V270            9
8    11442057  21518990        3    64511            9
9    11442057  21518990        5    65961            9


In [58]:
print(df_patients.columns)
print(df_admissions.columns)
print(df_edstays.columns)
print(df_icustays.columns)
print(df_triage.columns)
print(df_vitalsign.columns)
print(df_pyxis.columns)
print(df_medrecon.columns)
print(df_diagnoses.columns)

Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod'],
      dtype='object')
Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'ethnicity', 'edregtime', 'edouttime', 'insurance'],
      dtype='object')
Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime'], dtype='object')
Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime', 'outtime', 'los'],
      dtype='object')
Index(['subject_id', 'stay_id', 'triage_temperature', 'triage_heartrate',
       'triage_resprate', 'triage_o2sat', 'triage_sbp', 'triage_dbp',
       'triage_pain', 'triage_acuity', 'chiefcomplaint'],
      dtype='object')
Index(['subject_id', 'stay_id', 'charttime', 'ed_temperature', 'ed_heartrate',
       'ed_resprate', 'ed_o2sat', 'ed_sbp', 'ed_dbp', 'ed_rhythm', 'ed_pain'],
      dtype='object')
Index(['subject_id', 'stay_id', 'charttime', 'med_rn', 'name', 'gsn_rn',
       'gsn'],
      dtype='object'

## ED root table, demographic and outcomes

In [59]:
## Merging patients -> merging admissions -> merging triage -> master
df_master = merge_edstays_patients_on_subject(df_edstays ,df_patients)
df_master = merge_edstays_admissions_on_subject(df_master ,df_admissions)

In [60]:
## Adding age, mortality and ICU transfer outcome
df_master = add_age(df_master)
df_master = add_inhospital_mortality(df_master)
df_master = add_inhospital_mortality(df_master)
df_master = add_ed_mortality(df_master)
df_master = add_before_ed_mortality(df_master)
df_master = add_ed_los(df_master)
df_master = add_outcome_icu_transfer(df_master, df_icustays, icu_transfer_timerange)
df_master['outcome_hospitalization'] = ~pd.isnull(df_master['hadm_id'])
df_master['outcome_critical'] = df_master['outcome_inhospital_mortality'] | df_master[''.join(['outcome_icu_transfer_', str(icu_transfer_timerange), 'h'])]

# Sort Master table for further process
df_master = df_master.sort_values(['subject_id', 'intime']).reset_index()

# Filling subjects NA ethnicity, takes ~17s
df_master = fill_na_ethnicity(df_master)

## Health Utilization

In [61]:
## Generate past ED visits
df_master = generate_past_ed_visits(df_master, timerange=30)
df_master = generate_past_ed_visits(df_master, timerange=90)
df_master = generate_past_ed_visits(df_master, timerange=365)

In [62]:
## Oucome:  future ED revisit variables
df_master = generate_future_ed_visits(df_master, next_ed_visit_timerange)

In [63]:
## Generate past admissions
df_master = generate_past_admissions(df_master, df_admissions, timerange=30)
df_master = generate_past_admissions(df_master, df_admissions, timerange=90)
df_master = generate_past_admissions(df_master, df_admissions, timerange=365)

In [64]:
## Generate past icu visits
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=30)
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=90)
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=365)

In [65]:
## Generate numeric timedelta variables
df_master = generate_numeric_timedelta(df_master)

## Triage Information

In [66]:
## Mergining with triage table, Comment: revise the variable names? triage_*
df_master = merge_edstays_triage_on_subject(df_master, df_triage) ## note change to merge master 

In [67]:
## Encoding 10 chief complaints
df_master = encode_chief_complaints(df_master, complaint_dict)

## Comorbidities from diagnosis ICD

In [68]:
# This function takes about 10 min
df_master = commorbidity(df_master, df_diagnoses, df_admissions, timerange = 356*5)
extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v9')
extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v9_3digit')
extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v10')

Number of unique ICD codes v9:  5679
Number of unique ICD codes v9_3digit:  5571
Number of unique ICD codes v10:  7930


## ED Vital signs

In [69]:
df_master = merge_vitalsign_info_on_edstay(df_master, df_vitalsign, options=['last'])

## Medication

In [70]:
df_master = merge_med_count_on_edstay(df_master, df_pyxis)

In [71]:
df_master = merge_medrecon_count_on_edstay(df_master, df_medrecon)

In [72]:
df_master.shape

(448972, 119)

In [73]:
df_master.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,anchor_age,gender,anchor_year,dod,...,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
0,335314,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,52,F,2180,NaT,...,0,97.7,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0
1,335315,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,52,F,2180,NaT,...,1,97.9,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0
2,335316,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,52,F,2180,NaT,...,1,98.2,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0
3,335317,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,52,F,2180,NaT,...,1,99.0,96.0,18.0,97.0,86.0,45.0,NaN,8.0,14.0
4,335318,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,52,F,2180,NaT,...,1,98.1,91.0,18.0,99.0,98.0,60.0,NaN,6.0,7.0


## Generate ICD code list dataset for embedding

## Review the master dataset and output

In [74]:
#df_master.head(100).to_csv(os.path.join(output_path, 'master_dataset_part.csv'), index=False)
# Full dataset:
df_master.to_csv(os.path.join(output_path, 'master_dataset.csv'), index=False)